In [23]:
# # split the pdf into separate pages

# import PyPDF2
# from src.document_generation import setup_logger
# import logging

# logger = logging.getLogger('logger_name')
# if logger.hasHandlers():
#     logger.handlers.clear()  # Clear existing handlers to avoid duplicates
# logger.setLevel(logging.DEBUG)
# handler = logging.StreamHandler()
# formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
# handler.setFormatter(formatter)
# logger.addHandler(handler)
# logger.propagate = False

# # Input PDF file path
# input_pdf_path = "../input_data/Der Weltkrieg v7 East Front.pdf"
# output_folder = "../input_data/Der Weltkrieg v7"

# # Ensure the output folder exists
# os.makedirs(output_folder, exist_ok=True)

# # Open the PDF and split pages
# try:
#     with open(input_pdf_path, 'rb') as file:
#         pdf_reader = PyPDF2.PdfReader(file)
#         num_pages = len(pdf_reader.pages)
#         logger.info(f"Extracted {num_pages} pages from PDF")

#         for i, page in enumerate(pdf_reader.pages):
#             # Create a new PDF writer for each page
#             pdf_writer = PyPDF2.PdfWriter()
#             pdf_writer.add_page(page)

#             # Save the current page to a new file
#             output_file_path = os.path.join(output_folder, f"page_{i+1:03d}.pdf")
#             with open(output_file_path, 'wb') as output_file:
#                 pdf_writer.write(output_file)
            
#             logger.debug(f"Saved page {i+1} to {output_file_path}")
#             if i == 10:
#                 break

#     logger.info(f"All pages have been split and saved to {output_folder}")

# except Exception as e:
#     logger.error(f"An error occurred: {e}")


In [32]:
%%time
%load_ext autoreload
%autoreload 2

# Standard Python modules
import time 
import numpy as np
import os
import sys
import json
import re
import glob
import asyncio
import aiohttp
import openai
import logging
from typing import Dict, Tuple, List, Callable
from dotenv import load_dotenv
from pathlib import Path

# Decorator to log wall time
def log_execution_time(func: Callable):
    async def wrapper(*args, **kwargs):
        start = time.time()
        result = await func(*args, **kwargs)
        logger.info(f"Finished {func.__name__} in {time.time() - start:.2f} seconds.")
        return result
    return wrapper

def get_missing_keys(raw_german_texts):
    # print missing keys
    missing_keys = [ key for key in sorted(all_pagenos) if key not in raw_german_texts or len(raw_german_texts[key]) < 10 ]
    return missing_keys

# Load environment variables from .env file
env_path = Path('../.env')  # Adjust path if needed
load_dotenv(dotenv_path=env_path)

# # Get the root path of the project
sys.path.append(os.path.abspath(".."))

# Display and plotting
from IPython.display import display, HTML, clear_output

# Project imports
from src.utils import timeit, encode_image, plt, pylab
from src.processing import compute_log_spectrum_1d, extract_image_bbox, save_images
from src.api_requests import construct_payload_for_gpt, process_single_page
from src.document_generation import save_document

# Set notebook display width
display(HTML("<style>.container { width:90% !important; }</style>"))

# Print Python environment info
print('sys.executable:', sys.executable)
print('sys.version:', sys.version, '\n')

# Setup for PDF processing
foldername = "Der Weltkrieg v7"

# OpenAI API setup
openai.api_key = os.getenv("OPENAI_API_KEY")
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai.api_key}"
}

# --------------------
# Initialize variables
# --------------------
plotter = False
image_path = f"../input_data/{foldername}/*pdf"
fnames = sorted(glob.glob(image_path))
all_pagenos = [re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1) for fname in fnames]

# Storage for processed texts
raw_german_texts: Dict[str, str] = {}
german_texts: Dict[str, str] = {}
english_texts: Dict[str, str] = {}

# ------------------
# Configure logging
# ------------------
logger = logging.getLogger("time_logger")
if logger.hasHandlers():
    logger.handlers.clear()  # Clear existing handlers to avoid duplicates

logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

# Don't propagate message to parent loggers
logger.propagate = False 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


sys.executable: /Users/ozkansafak/code/fraktur/.venv/bin/python3
sys.version: 3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ] 

CPU times: user 3.1 ms, sys: 2.23 ms, total: 5.32 ms
Wall time: 4.38 ms


---
## PART 1
## Fraktur Translator (GPT-4o)

In [114]:
@log_execution_time
async def main(fnames, model_name="", semaphore_count=10, extract=True):
    semaphore = asyncio.Semaphore(semaphore_count)  # Adjust number based on API limits
    async def _process_page(fname: str) -> Tuple[str, Dict]:
        global raw_german_texts, german_texts, english_texts  # Explicitly declare globals
        async with semaphore:
            pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
            result = await process_single_page(fname, model_name, headers, plotter, pageno, extract) 
            return pageno, result
    
    # A list of coroutine objects. include only the unprocessed pages.
    keys = set(raw_german_texts.keys())
    tasks = []
    for fname in fnames:
        pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
        if pageno not in keys:
            tasks.append(_process_page(fname))
    logger.info(f"main: len(tasks): {len(tasks)} -- Processing tasks as they complete")
    
    # Process tasks as they complete
    for i, completed_task in enumerate(asyncio.as_completed(tasks)):
        try:
            pageno, (content, raw_german_text, german_text, english_text) = await completed_task
            raw_german_texts[pageno] = raw_german_text
            german_texts[pageno] = german_text
            english_texts[pageno] = english_text
            logger.info(f" {i:3d} of {len(tasks)} -- Successfully processed page:{pageno}")
        except Exception as e:
            logger.error(f"{i:3d} of {len(tasks)} -- Error processing a task: {e}")
    return 

# Run the async code
await main(fnames, model_name="gpt-4o-2024-08-06", semaphore_count=10)


2024-12-31 13:05:00,383 - INFO - main: len(tasks): 1 -- Processing tasks as they complete
2024-12-31 13:05:37,331 - INFO -    0 of 1 -- Successfully processed page:010
2024-12-31 13:05:37,333 - INFO - Finished main in 36.95 seconds.


---
## PART 2
##  Handle missing keys (Claude Sonnet)

In [126]:
# Check for unprocessed pages and print them to stdout.
def check_for_unprocessed_pages(fnames, missing_keys):
    # get corresponding filenames in missing keys
    missed_fnames = []
    for fname in fnames:
        pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
        if pageno in missing_keys:
            missed_fnames.append(fname)
    for item in missed_fnames:
        print(f"missed_fnames: {item}") 
        
    return missed_fnames

def delete_missing_keys(missing_keys, raw_german_texts):
    """
        deletes keys from `raw_german_texts` so they can be computed again,
    """
    for key in missing_keys:
        if key in raw_german_texts:
            del raw_german_texts[key]
            print(f'Deleted key:{key} from raw_german_texts')
        else:
            print(f'Attempted to delete key:{key} from raw_german_texts. but key not in raw_german_texts')

# ------------------------------------------------------------------------------------
# 1. Rerun the missing pages on Claude
# ------------------------------------------------------------------------------------

# Get missing keys based on empty raw_german_texts
missing_keys = set(get_missing_keys(raw_german_texts))
delete_missing_keys(missing_keys, raw_german_texts)
print((set(english_texts.keys())).difference(set(raw_german_texts.keys())))
missed_fnames = check_for_unprocessed_pages(fnames, missing_keys)

await main(missed_fnames, model_name="claude-3-5-sonnet-20241022", semaphore_count=1, extract=True)

# ------------------------------------------------------------------------------------
# 2. If there still are missing pages, run them without performing FFT based extraction. 
#    This time compute missing_keys based on 'english_texts'.
# ------------------------------------------------------------------------------------

# Now get missing keys based on empty english_texts
missing_keys = set(get_missing_keys(english_texts))
delete_missing_keys(missing_keys, english_texts)
print((set(raw_german_texts.keys())).difference(set(english_texts.keys()) ))

missed_fnames = check_for_unprocessed_pages(fnames, missing_keys)

await main(missed_fnames, model_name="claude-3-5-sonnet-20241022", semaphore_count=1, extract=False)


2024-12-31 13:19:34,434 - INFO - main: len(tasks): 2 -- Processing tasks as they complete


Deleted key:006 from raw_german_texts
Deleted key:405 from raw_german_texts
set()
{'006', '405'}
missed_fnames: ../input_data/Der Weltkrieg v7/page_006.pdf
missed_fnames: ../input_data/Der Weltkrieg v7/page_405.pdf


2024-12-31 13:20:08,379 - INFO -    0 of 2 -- Successfully processed page:006
2024-12-31 13:20:13,079 - INFO - Pageno: 405, "raw_german" section was not found
2024-12-31 13:20:13,081 - INFO - Pageno: 405, "german" section was not found
2024-12-31 13:20:13,082 - INFO - Pageno: 405, "english" section was not found
2024-12-31 13:20:13,089 - INFO -    1 of 2 -- Successfully processed page:405
2024-12-31 13:20:13,090 - INFO - Finished main in 38.66 seconds.
2024-12-31 13:20:13,094 - INFO - main: len(tasks): 0 -- Processing tasks as they complete
2024-12-31 13:20:13,095 - INFO - Finished main in 0.00 seconds.


Deleted key:405 from raw_german_texts
{'405'}
set()
missed_fnames: ../input_data/Der Weltkrieg v7/page_405.pdf


In [133]:
for pageno in missing_keys:
    english_texts[pageno] = '<Blank Page>'
    german_texts[pageno] = '<Blank Page>'
    print(f"{pageno}: <Blank Page>")
    
# Save json files and .docx files.
from src.document_generation import save_document

# save json outputs
if not os.path.exists(f'../output_data/{foldername}'):
    os.makedirs(f'../output_data/{foldername}')
with open(f'../output_data/{foldername}/english_texts.json', 'w') as f:
    json.dump(english_texts, f)
with open(f'../output_data/{foldername}/german_texts.json', 'w') as f:
    json.dump(german_texts, f)
with open(f'../output_data/{foldername}/raw_german_texts.json', 'w') as f:
    json.dump(raw_german_texts, f)

doc1, fname1 = save_document(german_texts, foldername, language='German')
doc2, fname2 = save_document(english_texts, foldername, language='English')


405: <Blank Page>


``` 
1.  Upload Input folder of pdfs to blob storage.
2.  Read file from s3.
3.  FFT in y -> (x_hi, x_lo), write half_cropped_image to s3
4.  FFT in x -> (y_hi, y_lo), write cropped_image to s3
5.  Read cropped image from s3 -> encode_image -> translate and transcribe -> JSON output

```

### Available models and pricing:
```
"gpt-4o-2024-08-06":
    "price_txt": "$2.50 / 1M input tokens"
    "price_img": "$0.001913 / 1500px^2"
    
"gpt-4o-mini-2024-07-18":
    "price_txt": "$0.150 / 1M input tokens"
    "price_img": "$0.003825 / 1500px^2"
    
```

---
## PART 3
## Load the German text and translate broken sentences.

In [136]:
with open(f'../output_data/{foldername}/raw_german_texts.json', 'r') as f:
    raw_german_texts = json.load(f)
with open(f'../output_data/{foldername}/german_texts.json', 'r') as f:
    german_texts = json.load(f)
with open(f'../output_data/{foldername}/english_texts.json', 'r') as f:
    english_texts = json.load(f)


In [247]:
# # Prepare input lists for german_texts and english_texts
# def extract_top_and_bottom_contents(german_texts, english_texts):
#     german_page_top_content = {}
#     german_page_bottom_content = {}
#     english_page_top_content = {}
#     english_page_bottom_content = {}
    
#     for pageno in all_pagenos:
#         # Extract from german_texts
#         german_bodies = re.findall(r'<body>(.*?)</body>', german_texts[pageno], re.DOTALL)
    
#         if german_bodies:
#             # all <body> sections excluding the very last one
#             german_page_top_content[pageno] = ['<body>' + item + '</body>' for item in german_bodies[:-1]]
#             # Last <body>
#             german_page_bottom_content[pageno] = '<body>' + german_bodies[-1] + '</body>'  
#         else:
#             german_page_top_content[pageno] = ['<body></body>']
#             german_page_bottom_content[pageno] = '<body></body>'
    
#         # Extract from english_texts
#         english_bodies = re.findall(r'<body>(.*?)</body>', english_texts[pageno], re.DOTALL)
    
#         if english_bodies:
#             # All <body> sections excluding the last one
#             english_page_top_content[pageno] = ['<body>' + item + '</body>' for item in english_bodies[:-1]]
#             # Last <body>
#             english_page_bottom_content[pageno] = english_bodies[-1]  
#         else:
#             english_page_top_content[pageno] = ['<body></body>']
#             english_page_bottom_content[pageno] = '<body></body>'
        
#     return (german_page_top_content, 
#             german_page_bottom_content,
#             english_page_top_content,
#             english_page_bottom_content)

# out = extract_top_and_bottom_contents(german_texts, english_texts)
# german_page_top_content, german_page_bottom_content, english_page_top_content, english_page_bottom_content = out

# initialize output dicts
english_texts_defragmented = {}
outputs = {}
payloads = {}
fragments_2 = {None:''}

In [248]:
from src.constants import FRAGMENTED_SENTENCES_PROMPT

def construct_payload_fragmented_sentences(
                        german_page_1: str,
                        german_page_2: str,
                        english_page_1_old: str,
                        german_page_1_top_fragment_to_be_ignored: str):

    model_name = "gpt-4o-2024-08-06"
    payload = {
        "model": model_name,
        "messages": [
          {
            "role": "system", 
            "content": "You are a World War II historian, who's bilingual in German and English "
              "You speak both languages with masterful efficiency and you're a professional translator from GERMAN to ENGLISH who "
              "stays loyal to both the style and the character of the original German text in your book translations."
          },
          {
            "role": "user",
            "content": [
              {
                  "type": "text",
                  "text": FRAGMENTED_SENTENCES_PROMPT.format(
                      german_page_1=german_page_1,
                      german_page_2=german_page_2,
                      english_page_1_old=english_page_1_old,
                      german_page_1_top_fragment_to_be_ignored=german_page_1_top_fragment_to_be_ignored
                  )
              },
            ]
          },
        ],
        "max_tokens": 6000,
        "temperature": 0.1
    }

    return payload 

In [259]:
import ipdb
import requests

def log_execution_time_synchronous(func: Callable):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        logger.info(f"Finished {func.__name__} in {time.time() - start:.2f} seconds.")
        return result
    return wrapper

def make_gpt_request_for_broken_sentences(headers: dict, payload: dict, pageno: str) -> dict:
    global english_texts_defragmented 

    model_name = "gpt-4o-2024-08-06"
    response = requests.post(
        "https://api.openai.com/v1/chat/completions",
        json=payload,
        headers=headers
    )
    result = response.json()
    response_content = result['choices'][0]['message']['content']
    english_texts_defragmented[pageno] = re.search(r'<english_page_1_new>(.*?)</english_page_1_new>', 
                                                   response_content, re.DOTALL).group(1)
    return response_content

@log_execution_time_synchronous
def main_broken_sentences(headers):
    global payloads, outputs, fragments_2
    for i in range(13): #range(len(all_pagenos)-1):
        prev_pageno = all_pagenos[i-1] if i > 0 else None
        pageno = all_pagenos[i]
        next_pageno = all_pagenos[i+1]
        if pageno in english_texts_defragmented.keys():
            logger.info(f'pageno: {pageno} already in english_texts_defragmented... skipping')
            continue

        for trial in range(1,4):
            try:
                logger.info(f"\nProcessing i:{i},  pageno: {pageno} {' ...trying again'+ str(trial) if trial>1 else ''}")
                payload = construct_payload_fragmented_sentences(
                    german_texts[pageno], 
                    german_texts[next_pageno], 
                    english_texts[pageno], 
                    fragments_2[prev_pageno] 
                )
                payloads[pageno] = payload['messages'][1]['content'][0]['text']
                response_content = make_gpt_request_for_broken_sentences(headers, payload, pageno)
                outputs[pageno] = response_content
                fragments_2[pageno] = re.search(r'<fragment_2>(.*?)</fragment_2>', response_content, re.DOTALL).group(1)
                if fragments_2[pageno].count('\n') > 10:
                    print('not accepting fragment_2. fragments_2[pageno].count("\\n")', fragments_2[pageno].count('\n'))
                    fragments_2[pageno] = '' 
                logger.info(f"next round's fragment_2: {fragments_2[pageno]}") 
                break
            except Exception as e:
                logger.error(f"Error processing. {e}. trial:{trial}, pageno: {pageno}")
                if trial == 3:
                    english_texts_defragmented[pageno] = ''
    english_texts_defragmented[next_pageno] = ''

main_broken_sentences(headers)


2024-12-31 15:17:32,552 - INFO - 
Processing i:0,  pageno: 001 
2024-12-31 15:17:36,729 - INFO - next round's fragment_2: 
2024-12-31 15:17:36,731 - INFO - 
Processing i:1,  pageno: 002 
2024-12-31 15:17:42,108 - INFO - next round's fragment_2: 
2024-12-31 15:17:42,109 - INFO - 
Processing i:2,  pageno: 003 
2024-12-31 15:17:50,095 - INFO - next round's fragment_2: Die im vorliegenden Band VII und dem in Bälde erscheinenden Band VIII geschilderte große Offensive im Osten bildet ein einheitliches Ganzes; der Rückblick auf den ganzen Zeitraum wird daher an den Schluß des VIII. Bandes gesetzt werden.
2024-12-31 15:17:50,097 - INFO - 
Processing i:3,  pageno: 004 
2024-12-31 15:17:55,215 - INFO - next round's fragment_2: 
2024-12-31 15:17:55,216 - INFO - 
Processing i:4,  pageno: 005 
2024-12-31 15:18:06,146 - INFO - next round's fragment_2: 
2024-12-31 15:18:06,148 - INFO - 
Processing i:5,  pageno: 006 
2024-12-31 15:18:19,501 - INFO - next round's fragment_2: 
2024-12-31 15:18:19,503 - 

In [ ]:
with open(f'../output_data/{foldername}/english_texts_defragmented.json', 'w') as f:
    json.dump(english_texts_defragmented, f)


In [173]:
del english_texts_defragmented['012']

In [188]:
for pageno in all_pagenos[:15]:
    print(pageno)
    print(english_texts_defragmented[pageno])
    

001

<header>The World War 1914 to 1918</header>
<body>
Edited in the Reich Archive
The Military Operations on Land
Seventh Volume
Published by E. S. Mittler & Son
Berlin in the Year 1931
</body>

002

<header>The Operations of the Year 1915</header>
<header>The Events in Winter and Spring</header>
<body>With forty maps and sketches</body>
<footer>Published by E. S. Mittler & Son</footer>
<footer>Berlin in the year 1931</footer>

003

<header>Introduction to the Seventh Volume.</header>
<body>The present Volume VII describes the military operations of the winter and spring of 1915.
To keep the scope of the complete work "The World War 1914—1918" within reasonable limits, a strict focus on the essentials was an unavoidable necessity. The presentation has aimed to highlight, by excluding less important events, the major lines of development of the war events, both in terms of operational warfare and the events at the battlefronts. While the depiction of the events of the first five month

KeyError: '014'

In [233]:
def get_english_texts_consolidated(english_texts, english_texts_defragmented):
    # consolidate the fragmented sentences
    english_texts_consolidated = {}
    for pageno in sorted(english_texts):
        try: 
            # get the last occurence of <body>. 
            # english_texts_defragmented[pageno] already carries the translation with fixed sentences for the <body> section of english_texts[pageno]
            # Find all <body> tags in english_texts[pageno]
            all_bodies = list(re.finditer(r'<body>(.*?)</body>', english_texts[pageno], re.DOTALL))

            if all_bodies:
                # Extract positions of the last <body> tag
                last_body = all_bodies[-1]  # Get the last occurrence
                i, j = last_body.span()
                
                updated = (english_texts[pageno][:i] 
                           + '<body>' + english_texts_defragmented[pageno] + '</body>' 
                           + english_texts[pageno][j:]
                          )
                english_texts_consolidated[pageno] = updated
            else:
                print(f"No <body> tags found for page {pageno}. Retaining original content.")
                english_texts_consolidated[pageno] = english_texts[pageno]english_texts_defragmented
        except:
            print(f'exception hit at {pageno}: Setting `english_texts_consolidated[pageno] = english_texts[pageno]`')
            english_texts_consolidated[pageno] = english_texts[pageno]

    return english_texts_consolidated

english_texts_consolidated = get_english_texts_consolidated(english_texts, english_texts_defragmented)


exception hit at 405: Setting `english_texts_consolidated[pageno] = english_texts[pageno]`


In [ ]:
pageno = '020' # fragment_2 is wrong

In [235]:
pageno  = '033'

search = re.search(r'<body>(.*?)</body>', english_texts[pageno], re.DOTALL) 
i, j = search.span() 
updated = english_texts[pageno][:i] + '<body>' + english_texts_defragmented[pageno] + '</body>' + english_texts[pageno][j:]
english_texts_consolidated[pageno] = updated


In [242]:
search.group(1)

'Five English and one Indian seemed recently to be divided into two armies. An agent report received by General v. Falkenhayn on January 27 expressed the prevailing belief in England that it was only a matter of gaining time to develop the military forces of the British Empire. Then they would win. The report also contained the suggestion that the appearance of Zeppelin airships over London would have an intimidating effect.\nThe Belgian army was in the process of rebuilding, but had not yet advanced far enough to consider its participation in an offensive as feasible.\nOverall, the views expressed by the Supreme Army Command in December about the growth of the French and English armies in the spring of 1915 seemed to be confirmed. Although there were initially no signs of the seven divisions of French reinforcements accounted for at the time, two English divisions not previously calculated had newly appeared. A major offensive seeking a decision against the enemies in the West had to 

In [ ]:
'029', '033', '64'

In [244]:
english_texts_consolidated[pageno]

'\n<pageno>21</pageno>\n<header>News about the Enemy.</header>\n<body>Five English and one Indian seemed recently to be divided into two armies. An agent report received by General v. Falkenhayn on January 27 expressed the prevailing belief in England that it was only a matter of gaining time to develop the military forces of the British Empire. Then they would win. The report also contained the suggestion that the appearance of Zeppelin airships over London would have an intimidating effect.\nThe Belgian army was in the process of rebuilding, but had not yet advanced far enough to consider its participation in an offensive as feasible.\nOverall, the views expressed by the Supreme Army Command in December about the growth of the French and English armies in the spring of 1915 seemed to be confirmed. Although there were initially no signs of the seven divisions of French reinforcements accounted for at the time, two English divisions not previously calculated had newly appeared. A major

In [234]:
for pageno in (english_texts_consolidated.keys()):
    print(f"pageno:{pageno}", english_texts_consolidated[pageno], '\n', '---'*3, '\n')

pageno:001 
<header>The World War 1914 to 1918</header>
<body>Edited in the Reich Archive
The Military Operations on Land
Seventh Volume
Published by E. S. Mittler & Son
Berlin in the Year 1931</body>
 
 --------- 

pageno:002 
<header>The Operations of the Year 1915</header>
<header>The Events in Winter and Spring</header>
<body>With forty maps and sketches</body>
<footer>Published by E. S. Mittler & Sohn</footer>
<footer>Berlin in the year 1931</footer>
 
 --------- 

pageno:003 
<header>Introduction to the Seventh Volume.</header>
<body>The present Volume VII describes the military operations of the winter and spring of 1915. To keep the scope of the complete work "The World War 1914—1918" within appropriate limits, strict limitation to the essential was an unavoidable necessity. The presentation has endeavored, by excluding less important events, to illustrate above all the major lines of development of the war events, both of the operational warfare and of the events at the battle

In [232]:
for pageno in english_texts_defragmented:
    search = re.search(r'<pageno>(.*?)</pageno>', english_texts_consolidated[pageno], re.DOTALL)
    if search:
        original_pageno = search.group(1) 
    else:
        original_pageno = ''
    print(f"pageno: {pageno}, original_pageno:{original_pageno}")


pageno: 001, original_pageno:
pageno: 002, original_pageno:
pageno: 003, original_pageno:
pageno: 004, original_pageno:
pageno: 005, original_pageno:
pageno: 006, original_pageno:
pageno: 007, original_pageno:
pageno: 008, original_pageno:
pageno: 009, original_pageno:
pageno: 010, original_pageno:
pageno: 011, original_pageno:
pageno: 012, original_pageno:
pageno: 013, original_pageno:
pageno: 014, original_pageno:
pageno: 015, original_pageno:
pageno: 016, original_pageno:
pageno: 017, original_pageno:
pageno: 018, original_pageno:
pageno: 019, original_pageno:
pageno: 020, original_pageno:
pageno: 021, original_pageno:
pageno: 022, original_pageno:
pageno: 023, original_pageno:
pageno: 024, original_pageno:
pageno: 025, original_pageno:
pageno: 026, original_pageno:
pageno: 027, original_pageno:
pageno: 028, original_pageno:
pageno: 029, original_pageno:General v. Falkenhayn supplemented these orders with a directive¹) on January 25 to all army high commands of the Western Front, in

In [48]:
body = re.search(r'<body>(.*?)</body>', german_texts['006'], re.DOTALL).group(1) 
body
german_page_contents['006']
german_texts['006']
print(german_page_contents['006'])



[Same content as above, maintaining all structure and formatting]



In [52]:
german_page_contents['006']

'\n[Same content as above, maintaining all structure and formatting]\n'

---
## Experiments